In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPool2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_dir ="input"
train_dir = os.path.join(base_dir, 'Training')
val_dir = os.path.join(base_dir, 'Testing')
test_dir = os.path.join(base_dir, 'Testing')

In [ ]:
epochs = 15
batch_size = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
#Generate batches of tensor image data
train_image_generator =  ImageDataGenerator(rescale = 1./255)
val_image_generator = ImageDataGenerator(rescale = 1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
#generates batches for train , val and test using directory path
train_data_generator = train_image_generator.flow_from_directory(
    train_dir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = batch_size,
    shuffle = True,
    class_mode = 'categorical'
)

val_data_generator = val_image_generator.flow_from_directory(
    directory = val_dir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = batch_size,
    shuffle = True,
    class_mode = 'categorical'
)

test_data_generator = test_image_generator.flow_from_directory(
    directory = test_dir,
    target_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = batch_size,
    class_mode = "categorical"
)

In [ ]:
image_batch, label_batch = next(iter(train_data_generator))
import matplotlib.pyplot as plt
class_names=list(train_data_generator.class_indices.keys())
plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i])
  label = label_batch[i].tolist().index(max(label_batch[i].tolist()))
  plt.title(class_names[label])
  plt.axis("off")

In [ ]:
for image_batch, label_match in train_data_generator:
    break
image_batch.shape, label_match.shape

In [ ]:
labels = train_data_generator.class_indices
print(labels)

In [ ]:
dataset_labels = sorted(train_data_generator.class_indices.items())
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(IMG_HEIGHT,IMG_WIDTH,3),
                                              include_top=False, 
                                              weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
#compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
#check model summary
model.summary()

In [ ]:
history = model.fit_generator(train_data_generator, 
                    epochs=epochs, 
                    validation_data=val_data_generator)

In [ ]:
#find loss and accuracy on test data
test_loss, test_acc = model.evaluate_generator(test_data_generator)
print('test acc:', test_acc)

In [ ]:
model.save("brain_model_pretrain_v1.h5")

In [ ]:
#now load the above save model
brain_model = tf.keras.models.load_model("brain_model_pretrain_v1.h5")
print(brain_model)

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
#start training layer from block 5
base_model.trainable = True
set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
#compile the model
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
#check accuracy
model.summary()

In [ ]:
#start training
history_fine = model.fit_generator(train_data_generator, 
                         epochs=10,
                         validation_data=val_data_generator)

In [ ]:
model.save("models/brain_model_finetune.h5")
